In [53]:
import json #for your mc json files
import os # for env
from langchain_groq import ChatGroq #groq inf engine 
from langchain_google_genai import ChatGoogleGenerativeAI #google ai studio
from langchain_core.prompts import ChatPromptTemplate #for input output prompt 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough #Chatrunnable interface 
from langchain_core.messages import HumanMessage, AIMessage #types of messages tht needDs to be passed to The LLms 


In [54]:
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")
print("api loaded")

api loaded


In [55]:
from langchain_core.documents import Document #provess docs
from langchain.text_splitter import RecursiveCharacterTextSplitter #for chunks
from langchain_google_genai import GoogleGenerativeAIEmbeddings #for google embeddings model 001
from langchain_community.vectorstores import FAISS #retrieval
from langchain.chains import create_history_aware_retriever, create_retrieval_chain #fro chain histroy
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.embeddings import OllamaEmbeddings # Using Ollama for local embeddings
from langchain_ollama import OllamaEmbeddings 


In [56]:
#function to choose files 
#btc or eth 
# a full function.
def create_retriever_for_coin(coin_choice: str):

    if coin_choice.lower() == 'btc':
        file_path = r"C:\Users\ishaa\OneDrive\Desktop\AgenticAi\Agentic-Ai\Udmy\Gayrav-and-team\btc_train.json"
        coin_name = 'Bitcoin'
    elif coin_choice.lower() == 'eth':
        file_path = r"c:\Users\ishaa\OneDrive\Desktop\AgenticAi\Agentic-Ai\Udmy\Gayrav-and-team\eth_train.json"
        coin_name = 'Ethereum'
    else:
        print("Not a Valid choice")
        return None
    

      # Load and process the selected JSON file whatever the user chose
    documents = []
        #bug ( to be changed after protyp )
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for date, daily_data in data.items():
        news_text = " ".join(daily_data['news'])
        metadata = {"date": date, "price": daily_data['prices'], "coin": coin_name}
        doc = Document(page_content=news_text, metadata=metadata)
        documents.append(doc)




            # chiunking into smaller 
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunked_docs = text_splitter.split_documents(documents)

    # Creating the embeddings and the vector store from facebook
    embeddings = OllamaEmbeddings(model="embeddinggemma:300m") ## frommmm OLLLAMAAAAAAA GEMMAAAAA
    vector_store = FAISS.from_documents(chunked_docs, embedding=embeddings)
    
    # Create and return the retriever
    retriever = vector_store.as_retriever(search_kwargs={"k": 5}) # k is the top best searches from as_retriever 

    print(f" Retriever for {coin_name} created")
    return retriever

In [57]:
retriever = create_retriever_for_coin(user_selection)


 Retriever for Bitcoin created


In [58]:
# Add this to your notebook:
def create_chat_chain(retriever):
    # Initialize your LLM (Groq or Gemini)
    llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.3)
    
    # Create system prompt for crypto analysis
    system_prompt = """
    You are a crypto analysis expert. Use the provided context to answer questions about cryptocurrency prices, news, and market trends.
    
    Guidelines:
    Provide specific insights based on the retrieved data
     Mention relevant dates and price movements
    Explain your reasoning clearly
    If you don't have enough info, say so
    
    Context: {context}
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{input}")
    ])
    
   
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain) #chain wull be invoked from here
    
    return retrieval_chain


In [59]:
# Add this cell after your retriever setup:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Create conversational memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Your existing Groq/Gemini LLM
    retriever=retriever,  # Your existing FAISS retriever
    memory=memory,
    verbose=True
)


In [60]:
# Fix the import issue first
from langchain_core.prompts import MessagesPlaceholder

# Your working LLM setup
llm = ChatGroq(model_name="qwen/qwen3-32b")

# Create contextualization prompt (for history awareness)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", "Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# Create history-aware retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

print("Histry will be implemented when Streamlit app will be initialised")


Histry will be implemented when Streamlit app will be initialised


In [61]:
# Create the QA system prompt
qa_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a crypto analysis expert specialized in Bitcoin and Ethereum. 
    
    Use the retrieved context to provide insights about:
     Price trends and market movements
     News sentiment analysis  
     Investment recommendations
     Market predictions based on current factors
    
    Always explain your reasoning and cite specific dates/prices when available.
    
    Context: {context}"""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
])

# Create the question-answer chain invoked will be by the function crypto-chat
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

print("Chain Created")


Chain Created


In [ ]:
# Simple function to chat with your crypto AI
def crypto_chat(question):
    response = rag_chain.invoke({
        "input": question,
        "chat_history": []  #abhi chord di h , decaprivate hogya h kuch langchain me Dekhna padega 
    })
    return response["answer"]

# Test it!
print(" crupto ai ready for eth and btc!")
print("------------------------------- ----------GAY-RAV--------------------------------------------------")

# Demo questions





 crupto ai ready for eth and btc!



In [66]:
question = "What's the recent sentiment around Bitcoin according to the data present in my context and files?"
print(f"Question: {question}")
print("Analyzing Bitcoin sentiment from news data...")
print("-" * 60)

response = crypto_chat(question)
print(f"AI Analysis: {response}")
print("=" * 60)


Question: What's the recent sentiment around Bitcoin according to the data present in my context and files?
Analyzing Bitcoin sentiment from news data...
------------------------------------------------------------
AI Analysis: <think>
Okay, let me try to figure out the recent sentiment around Bitcoin based on the given context. First, I need to look through each of the provided sentences and note their sentiments and key points.

The first sentence says that Bitcoin revenue is down 7% due to a price decline, which has a negative sentiment. That's a clear negative point. Then there's Block Inc.'s share price jumping in after-hours trading after outperforming expectations, but their Bitcoin revenue dipped because of the price drop. Still, the sentiment here is negative because the price decline is affecting revenue.

Next, there's a positive sentiment about Bitcoin being used during the banking crisis, which is a positive aspect. Greenpeace's "Skull of Satoshi" artwork was initially neg

In [67]:
question = "Should I invest in Bitcoin based on recent trends?"
print(f"Question: {question}")
print("Analyzing Bitcoin investment potential...")
print("-" * 60)

response = crypto_chat(question)
print(f"AI Recommendation: {response}")
print("=" * 60)


Question: Should I invest in Bitcoin based on recent trends?
Analyzing Bitcoin investment potential...
------------------------------------------------------------
AI Recommendation: <think>
Okay, the user is asking whether they should invest in Bitcoin based on recent trends. Let me start by analyzing the context provided.

First, there are a few key points here. Bitcoin's revenue is down 7% due to a price decline, which is negative. However, Block Inc's stock went up after hours, so maybe that's a separate point. Then there's the positive sentiment about the next bull run coming from Asia and America needing to embrace crypto. The market cap flipping Visa three times is positive, showing recovery after the FTX collapse. Pantera Capital says the lows are behind us and it's a year of rebuilding trust. Also, on-chain indicators are bullish, suggesting a potential bull market. On the negative side, there's the Bitcoin ATM scams and criticism of Bitcoin NFTs.

I need to balance these out.

In [68]:
question = "How does Ethereum compare to Bitcoin right now?"
print(f"Question: {question}")
print("Comparing BTC vs ETH market positions...")
print("-" * 60)

response = crypto_chat(question)
print(f"AI Comparison: {response}")
print("=" * 60)


Question: How does Ethereum compare to Bitcoin right now?
Comparing BTC vs ETH market positions...
------------------------------------------------------------
AI Comparison: <think>
Okay, the user is asking how Ethereum compares to Bitcoin right now. Let me start by recalling the context provided. The context mentions Bitcoin's revenue dropping 7% due to crypto price declines, but there's positive sentiment about the next bull run coming from Asia and Bitcoin's market cap flipping Visa. Also, Pantera Capital is optimistic about the next bull market cycle. Ethereum isn't directly mentioned in the context, so I need to infer based on general knowledge and the Bitcoin situation.

First, I should consider the price performance. The user might know Bitcoin's recent price movements, but how is Ethereum doing? If Bitcoin is down 7%, maybe Ethereum is down as well, but perhaps not by the same margin. I should check if there's any correlation between Bitcoin and Ethereum in the current market 

In [69]:
question = "What are the prices of ETH? Provide a detailed summary with examples."
print(f"Question: {question}")
print("Analyzing Ethereum price patterns and trends...")
print("-" * 60)

response = crypto_chat(question)
print(f"AI Price Analysis: {response}")
print("=" * 60)


Question: What are the prices of ETH? Provide a detailed summary with examples.
Analyzing Ethereum price patterns and trends...
------------------------------------------------------------
AI Price Analysis: <think>
Okay, the user is asking for the current price of Ethereum (ETH) and a detailed summary with examples. Let me start by recalling the context provided. The context includes information about Bitcoin's price trends, market movements, and some positive sentiment indicators. However, the user specifically asked about Ethereum, which wasn't directly mentioned in the context. 

First, I need to check if there's any ETH-related information in the context. The context talks about Bitcoin's revenue decline due to price drops, Block Inc.'s performance, and positive signs for Bitcoin's bull market. There's also mention of Binance expanding in Georgia and Asia's role in the next crypto bull run. But nothing explicit about Ethereum's price.

Since the context doesn't provide ETH price d

In [75]:
question = "How does banking crisis affect Bitcoin as safe haven? also compare the prices present in my file context and the provide a simple ypour knowledge based prediction of btc in 2025 and 2026 and 2027 at the end provide a number ( amount ) of prediction at the end as conclusion"
print(f"Question: {question}")
print("Analyzing Ethereum price patterns and trends...")
print("-" * 60)

response = crypto_chat(question)
print(f"AI Price Analysis: {response}")
print("=" * 60)


Question: How does banking crisis affect Bitcoin as safe haven? also compare the prices present in my file context and the provide a simple ypour knowledge based prediction of btc in 2025 and 2026 and 2027 at the end provide a number ( amount ) of prediction at the end as conclusion
Analyzing Ethereum price patterns and trends...
------------------------------------------------------------
AI Price Analysis: <think>
Okay, let's tackle this query step by step. The user is asking about the impact of the banking crisis on Bitcoin as a safe haven and wants a price prediction for 2025, 2026, and 2027 based on the provided context.

First, I need to address how the banking crisis affects Bitcoin. From the context, there's mention of a U.S. bank run and its impact on Bitcoin liquidity. Typically, during banking crises, investors seek safe havens like gold or Bitcoin. However, the context also shows mixed signals. For instance, Bitcoin's price is fluctuating, sometimes dropping due to inflatio